In [ ]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform_train, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform_test, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 26421880/26421880 [00:02<00:00, 12435642.49it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 207542.04it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3910995.11it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5829448.43it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:


class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(64*13*13, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 64*13*13)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

model = FashionCNN()


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
epochs = 275  # Increase the number of epochs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, epochs, running_loss/len(train_loader)))


Epoch [1/275], Loss: 0.5818
Epoch [2/275], Loss: 0.3957
Epoch [3/275], Loss: 0.3499
Epoch [4/275], Loss: 0.3206
Epoch [5/275], Loss: 0.3043
Epoch [6/275], Loss: 0.2873
Epoch [7/275], Loss: 0.2746
Epoch [8/275], Loss: 0.2666
Epoch [9/275], Loss: 0.2550
Epoch [10/275], Loss: 0.2488
Epoch [11/275], Loss: 0.2429
Epoch [12/275], Loss: 0.2370
Epoch [13/275], Loss: 0.2290
Epoch [14/275], Loss: 0.2243
Epoch [15/275], Loss: 0.2212
Epoch [16/275], Loss: 0.2151
Epoch [17/275], Loss: 0.2147
Epoch [18/275], Loss: 0.2100
Epoch [19/275], Loss: 0.2076
Epoch [20/275], Loss: 0.2008
Epoch [21/275], Loss: 0.1989
Epoch [22/275], Loss: 0.1998
Epoch [23/275], Loss: 0.1951
Epoch [24/275], Loss: 0.1899
Epoch [25/275], Loss: 0.1934
Epoch [26/275], Loss: 0.1916
Epoch [27/275], Loss: 0.1864
Epoch [28/275], Loss: 0.1836
Epoch [29/275], Loss: 0.1861
Epoch [30/275], Loss: 0.1794
Epoch [31/275], Loss: 0.1788
Epoch [32/275], Loss: 0.1736
Epoch [33/275], Loss: 0.1725
Epoch [34/275], Loss: 0.1752
Epoch [35/275], Loss: 0

In [ ]:

model.eval()
correct = 0
total = 0
y_true = []
y_pred = []

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        y_true.extend(labels.tolist())
        y_pred.extend(predicted.tolist())

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
print(classification_report(y_true, y_pred))

Accuracy of the network on the 10000 test images: 91 %
              precision    recall  f1-score   support

           0       0.83      0.92      0.87      1000
           1       0.99      0.98      0.99      1000
           2       0.80      0.93      0.86      1000
           3       0.93      0.93      0.93      1000
           4       0.90      0.79      0.84      1000
           5       0.99      0.98      0.99      1000
           6       0.81      0.70      0.75      1000
           7       0.96      0.98      0.97      1000
           8       0.99      0.98      0.99      1000
           9       0.97      0.97      0.97      1000

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000

